In [ ]:
#| default_exp lab

In [ ]:
#| export

import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np
from chewc.sim import *
import torch

class SelectionIntensityEnvironment(gym.Env):
    def __init__(self, SP, config):
        super(SelectionIntensityEnvironment, self).__init__()
        self.SP = SP
        self.config = config  # Store the config
        self.current_generation = 0
        self.max_generations = SP.max_generations
        # Get action space bounds from config, with defaults if not provided
        
        self.action_low = config.get('action_low', 0.05)
        self.action_high = config.get('action_high', 0.95)
        
        # Update action space with custom bounds
        self.action_space = gym.spaces.Box(
            low=np.array([-1]), 
            high=np.array([1]), 
            dtype=np.float32
        )
        
        # Update observation space based on config
        obs_config = config['observation_config']['remaining_proportion']
        self.observation_space = gym.spaces.Box(
            low=obs_config['low'],
            high=obs_config['high'],
            shape=(1,),
            dtype=np.float32
        )
        # logging
        self.action_values = []
        self.genetic_variance = []
        self.max_breeding_values = []
        self.final_generations = []
        self.episode_count = 0
        self.rewards = []
        self.episode_reward = 0
        
        #config
        self.config =config
        
    def _get_obs(self):
        remaining_proportion = 1 - (self.current_generation / self.max_generations)
        return np.array([remaining_proportion], dtype=np.float32)


    def _get_info(self):
        return {
            "max_phenotype": self.population.breeding_values.max().cpu().item(),
            "genetic_variance": self.population.breeding_values.var().cpu().item(),
            "current_generation": self.current_generation
        }

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.population = self.SP.founder_pop
        self.phenotype = phenotype(self.population, self.SP.T, self.SP.h2)
        self.current_generation = 0
        self.episode_reward = 0
        observation = self._get_obs()
        info = self._get_info()
        
        return observation, info

    def step(self, action):
        
        # Map the action from [-1, 1] to [action_low, action_high]
        action = scale_values(action, to_range=(self.action_low, self.action_high))

        total_selected = max((2,int(action * self.population.size)))
        selected = torch.topk(self.population.phenotypes, total_selected).indices
        self.population = create_pop(self.SP.G, random_crosses(self.population.haplotypes[selected], self.SP.pop_size))
        self.phenotype = phenotype(self.population, self.SP.T, self.SP.h2)
        self.current_generation += 1

        observation = self._get_obs()
        info = self._get_info()
        info['normalized_action'] = action
        
        terminated = self.current_generation > self.SP.max_generations
        #REWARD
        if self.config.get('sparse_reward', False):  # Use .get() with a default value
            reward = 0 if not terminated else float(self.population.breeding_values.max())
        else:
            reward = float(self.population.breeding_values.max())
        self.episode_reward += reward

        if terminated:
            info['final_generation'] = {
            "max_phenotype": self.population.breeding_values.max().cpu().item(),
            "genetic_variance": self.population.breeding_values.var().cpu().item(),
            "current_generation": self.current_generation
            }

        return observation, reward, bool(terminated), False, info



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()